## Importing Required Libraries

In [1]:
from glob import glob
from sklearn.model_selection import GroupKFold
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import torch
from torch.utils.data import Dataset, DataLoader
import torch.nn as nn
import os
import random
import torchvision.transforms as transforms
from PIL import Image
from matplotlib.pyplot import imshow
from sklearn.metrics import *
import torch.nn.functional as F
import transformers
from transformers import AdamW, get_linear_schedule_with_warmup, get_constant_schedule
import warnings
warnings.filterwarnings("ignore")

wandb: WARNING W&B installed but not logged in.  Run `wandb login` or set the WANDB_API_KEY env variable.


## Importing EfficientNet Pretrained Model

In [2]:
!pip install efficientnet_pytorch
from efficientnet_pytorch import EfficientNet

  Created wheel for efficientnet-pytorch: filename=efficientnet_pytorch-0.6.3-py3-none-any.whl size=12419 sha256=4e3aac74008befb736bbd803511db4edeabf704d9f113f82fe1e0d414f477118
  Stored in directory: /root/.cache/pip/wheels/90/6b/0c/f0ad36d00310e65390b0d4c9218ae6250ac579c92540c9097a
Successfully built efficientnet-pytorch


## Defining The Model

In [3]:
def get_net():
    net = EfficientNet.from_pretrained('efficientnet-b2')
    net._fc = nn.Linear(in_features=1408, out_features=4, bias=True)
    return net

mx = get_net()
model = mx.to(torch.device('cuda:0'))

Downloading: "https://github.com/lukemelas/EfficientNet-PyTorch/releases/download/1.0/efficientnet-b2-8bb594d6.pth" to /root/.cache/torch/checkpoints/efficientnet-b2-8bb594d6.pth



Loaded pretrained weights for efficientnet-b2


## Loading saved weights and Evaluating

In [4]:
checkpoint = torch.load('../input/alaska/model4.bin')
model.load_state_dict(checkpoint)
model.eval()

EfficientNet(
  (_conv_stem): Conv2dStaticSamePadding(
    3, 32, kernel_size=(3, 3), stride=(2, 2), bias=False
    (static_padding): ZeroPad2d(padding=(0, 1, 0, 1), value=0.0)
  )
  (_bn0): BatchNorm2d(32, eps=0.001, momentum=0.010000000000000009, affine=True, track_running_stats=True)
  (_blocks): ModuleList(
    (0): MBConvBlock(
      (_depthwise_conv): Conv2dStaticSamePadding(
        32, 32, kernel_size=(3, 3), stride=[1, 1], groups=32, bias=False
        (static_padding): ZeroPad2d(padding=(1, 1, 1, 1), value=0.0)
      )
      (_bn1): BatchNorm2d(32, eps=0.001, momentum=0.010000000000000009, affine=True, track_running_stats=True)
      (_se_reduce): Conv2dStaticSamePadding(
        32, 8, kernel_size=(1, 1), stride=(1, 1)
        (static_padding): Identity()
      )
      (_se_expand): Conv2dStaticSamePadding(
        8, 32, kernel_size=(1, 1), stride=(1, 1)
        (static_padding): Identity()
      )
      (_project_conv): Conv2dStaticSamePadding(
        32, 16, kernel_size=

## Retrieving Test Images from Directory

In [5]:
class DatasetSubmissionLoader(Dataset):
    
    def __init__(self, image_names, transforms = None):
        super().__init__()
        self.image_names = image_names
        self.transforms = transforms
        
    def __getitem__(self, index: int):
        img_name = self.image_names[index]
        image = Image.open(f'../input/alaska2-image-steganalysis/Test/{img_name}')
        if self.transforms:
            image = self.transforms(image)
        return img_name, image
    
    
    def __len__(self) -> int:
        return self.image_names.shape[0]
    
    def get_names(self):
        return self.image_names

## Applying TTA (Test Time Augmentation):

In [6]:
def test_transforms():
    return transforms.Compose([
        transforms.RandomHorizontalFlip(p=0.5),
        transforms.RandomVerticalFlip(p=0.5),
        transforms.Resize((512,512)),
        transforms.ToTensor()])

## Getting Data into DataFrame

In [7]:
dataset = DatasetSubmissionLoader(
    image_names = np.array([path.split('/')[-1] for path in glob('../input/alaska2-image-steganalysis/Test/*.jpg')]),
    transforms = test_transforms())

In [8]:
data_loader = DataLoader(dataset, batch_size = 8, shuffle = False, num_workers = 4, drop_last = False)

## Predicting the Results

In [9]:
result = {'Id':[], 'Label': []}
for step, (image_names, images) in enumerate(data_loader):
    print(step, end = '\r')
    
    device = device = torch.device('cuda:0')
    images = images.to(device, dtype = torch.float32)
    y_pred = model(images)
    y_pred = 1 - F.softmax(y_pred, dim = 1).data.cpu().numpy()[:,0]
    
    result['Id'].extend(image_names)
    result['Label'].extend(y_pred)

## Getting Submission File Ready

In [10]:
submission = pd.DataFrame(result)
submission.to_csv('submission_file_alaska.csv', index = False)